In [50]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import regex as re
import os
import seaborn as sns
from openpyxl import load_workbook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%reload_ext rpy2.ipython

In [51]:
os.chdir('/Users/arvind/Documents/Work/Zoho WorkDrive (Educational Initiatives Private Limited)/My Folders/Projects/CSF/Analysis/Code/DQA/')

<h1>
    Import raw data files
</h1>

In [52]:
#Importing all scored files
mp_literacy_scored = pd.read_excel("./2022_11_08_mp_literacy_total_scores.xlsx", index_col=[0])
mp_numeracy_scored = pd.read_excel("./2022_11_08_mp_numeracy_total_scores.xlsx", index_col=[0])
up_literacy_scored = pd.read_excel("./2022_11_08_up_literacy_total_scores.xlsx", index_col=[0])
up_numeracy_scored = pd.read_excel("./2022_11_08_up_numeracy_total_scores.xlsx", index_col=[0])

In [53]:
all(col in up_literacy_scored.columns for col in mp_literacy_scored.columns)
all(col in up_numeracy_scored.columns for col in mp_numeracy_scored.columns)

True

In [54]:
#Merging state files
literacy_scored = pd.concat([mp_literacy_scored,up_literacy_scored],join='inner')
numeracy_scored = pd.concat([mp_numeracy_scored,up_numeracy_scored],join='inner')

In [55]:
literacy_scored['school_details.State_label'].unique()

array(['Madhya Pradesh', 'Madhya_Pradesh', 'Uttar Pradesh',
       'Uttar_Pradesh'], dtype=object)

In [56]:
literacy_scored['school_details.State_label'] = literacy_scored['school_details.State_label'].replace('Madhya_Pradesh','Madhya Pradesh')
literacy_scored['school_details.State_label'] = literacy_scored['school_details.State_label'].replace('Uttar_Pradesh','Uttar Pradesh')
numeracy_scored['school_details.State_label'] = numeracy_scored['school_details.State_label'].replace('Madhya_Pradesh','Madhya Pradesh')
numeracy_scored['school_details.State_label'] = numeracy_scored['school_details.State_label'].replace('Uttar_Pradesh','Uttar Pradesh')
                                                                                                    

In [57]:
#Output for Tableau Workbook
literacy_scored.to_excel("../Results/literacy_scored.xlsx")
numeracy_scored.to_excel("../Results/numeracy_scored.xlsx")

In [58]:
literacy_scored.columns.to_list()

['tabletUserName',
 'assessment_date',
 'school_details.State_label',
 'school_details.District_label',
 'school_details.Block_label',
 'school_details.School_label',
 'school_details.UDISE_cd_label',
 'SI_std_name',
 'student_age',
 'student_gender',
 'literacy1_total',
 'literacy2_total',
 'literacy3_total',
 'literacy4_ut_total',
 'literacy4_tt_total',
 'literacy4_tt_time_taken',
 'literacy4_tt_cpm',
 'literacy5_ut_total',
 'literacy5_tt_total',
 'literacy5_tt_time_taken',
 'literacy5_tt_cpm',
 'literacy6_total',
 'literacy6_tt_time_taken',
 'literacy6_tt_cpm',
 'literacy7_total',
 'literacy7_tt_time_taken',
 'literacy7_tt_cpm',
 'literacy8_reading_total',
 'literacy8_comprehension_total',
 'literacy9a_total',
 'literacy9b_total']

In [63]:
literacy_scored.groupby(['student_gender'])['literacy8_comprehension_total'].sum()

student_gender
Female    23
Male      71
Name: literacy8_comprehension_total, dtype: int64

In [ ]:
#renaming variables for ease and use in R
literacy_scored = literacy_scored.rename(columns = {'school_details.State_label':'State',
                                  'school_details.District_label':'District',
                                  'school_details.Block_label':'Block',
                                  'school_details.School_label':'School',
                                  'school_details.UDISE_cd_label':'UDISE',
                                  'SI_std_name':'Name',
                                  'student_age':'Age',
                                  'student_gender':'Gender'})
numeracy_scored = numeracy_scored.rename(columns = {'school_details.State_label':'State',
                                  'school_details.District_label':'District',
                                  'school_details.Block_label':'Block',
                                  'school_details.School_label':'School',
                                  'school_details.UDISE_cd_label':'UDISE',
                                  'SI_std_name':'Name',
                                  'student_age':'Age',
                                  'student_gender':'Gender'})

In [ ]:
literacy_scored.head()

In [ ]:
literacy_scored['id'] = literacy_scored.index
numeracy_scored['id'] = numeracy_scored.index

In [ ]:
%%R -i literacy_scored

require(dplyr)
require(Hmisc)
require(sqldf)
sqldf("select State, Gender, count(distinct id) as ct from literacy_scored group by State, Gender")

In [ ]:
# Check for gender distribution
literacy_scored.groupby(['State','Gender'])['id'].count()

In [ ]:
# check for student responses per school greater than 12
chk1 = pd.DataFrame(literacy_scored.groupby(['UDISE'])['id'].count())
chk1['UDISE'] = chk1.index
chk1 = pd.DataFrame(chk1.groupby(['id'])['UDISE'].count())
chk2 = pd.DataFrame(numeracy_scored.groupby(['UDISE'])['id'].count())
chk2['UDISE'] = chk2.index
chk2 = pd.DataFrame(chk2.groupby(['id'])['UDISE'].count())
chk1.merge(chk2,how='left',on='id')

In [ ]:
chk = pd.DataFrame(literacy_scored.groupby(['UDISE','Name','Age','Gender'])['id'].count())
chk['UDISE'] = chk.index
chk.groupby(['id'])['UDISE'].count()


In [ ]:
literacy_scored[literacy_scored['literacy9b_total']>7].groupby(['State','District'])['Name'].count()